In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import display,Markdown,update_display
import ollama

In [2]:
OLLMAN_API_URL = "http://localhost:11434/api/chat"
MODEL = "llama3.2"


In [24]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

from urllib.parse import urljoin

class Website:
    def __init__(self, url):
        self.url = url
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
        except requests.RequestException as e:
            self.title = "İstek başarısız"
            self.text = str(e)
            self.links = []
            return

        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else 'Başlık bulunamadı'

        if soup.body:
            for irrelevant in soup(['script', 'style', 'header', 'footer', 'nav', 'aside']):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = 'Sayfa içeriği bulunamadı'

        raw_links = [link.get('href') for link in soup.find_all('a')]
        # Sadece http/https ile başlayan linkleri al
        self.links = list(set(urljoin(self.url, l) for l in raw_links if l and l.startswith(('http://', 'https://'))))

    def get_content(self):
        return f"Sayfa Başlığı:\n{self.title}\nSayfa İçeriği:\n{self.text}\n\n"


In [4]:
deponext=Website("https://www.deponext.com.tr")
deponext.links

['https://wa.me/+905373234343',
 'mailto:info@deponext.com',
 'tel:+905373234343',
 'https://www.google.com/maps/place/Cam+Pak+otopark+ve+y%C4%B1kama/@41.0655793,29.0040037,21z/data=!4m14!1m7!3m6!1s0x14cab654199f9f09:0xc56988360e82a6aa!2zR2F5cmV0dGVwZSwgVmVmYWJleSBTb2thxJ_EsSBObzoyNSBEOjEsIDM0MzQ5IEJlxZ9pa3RhxZ8vxLBzdGFuYnVs!3b1!8m2!3d41.0656118!4d29.0042239!3m5!1s0x14cab7e0e90d1005:0x1bfb0e4fa8b47755!8m2!3d41.0656188!4d29.0039287!16s%2Fg%2F11y1dv3j2v?entry=ttu&g_ep=EgoyMDI1MDYxNy4wIKXMDSoASAFQAw%3D%3D',
 'mailto:info@deponext.com',
 'tel:+905373234343',
 'https://www.google.com/maps/place/Cam+Pak+otopark+ve+y%C4%B1kama/@41.0655793,29.0040037,21z/data=!4m14!1m7!3m6!1s0x14cab654199f9f09:0xc56988360e82a6aa!2zR2F5cmV0dGVwZSwgVmVmYWJleSBTb2thxJ_EsSBObzoyNSBEOjEsIDM0MzQ5IEJlxZ9pa3RhxZ8vxLBzdGFuYnVs!3b1!8m2!3d41.0656118!4d29.0042239!3m5!1s0x14cab7e0e90d1005:0x1bfb0e4fa8b47755!8m2!3d41.0656188!4d29.0039287!16s%2Fg%2F11y1dv3j2v?entry=ttu&g_ep=EgoyMDI1MDYxNy4wIKXMDSoASAFQAw%3D%3D',
 'https://wa

In [5]:
link_system_prompt = (
    "Size bir web sayfasında bulunan linklerin listesi verilmektedir. "
    "Bu linklerden hangilerinin şirket broşürü için en uygun olduğunu belirleyebilirsiniz, "
    "örneğin Hakkımızda sayfası, Şirket sayfası veya Kariyer/İş ilanları sayfaları.\n"
    "Yanıtınızı JSON formatında verin. Örnek format aşağıdadır:\n"
    "{\n"
    '    "links": [\n'
    '        {"type": "about page", "url": "https://full.url/goes/here/about"},\n'
    '        {"type": "careers page", "url": "https://another.full.url/careers"}\n'
    "    ]\n"
    "}"
)


In [6]:
print(link_system_prompt)

Size bir web sayfasında bulunan linklerin listesi verilmektedir. Bu linklerden hangilerinin şirket broşürü için en uygun olduğunu belirleyebilirsiniz, örneğin Hakkımızda sayfası, Şirket sayfası veya Kariyer/İş ilanları sayfaları.
Yanıtınızı JSON formatında verin. Örnek format aşağıdadır:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}


In [7]:
def get_links_user_prompt(website):
    user_prompt = (
        f"{website.url} sitesindeki linklerin listesi aşağıdadır. "
        "Bu linklerden hangilerinin şirket broşürü için uygun olduğunu seçin, "
        "örneğin Hakkımızda, Şirket veya Kariyer sayfaları. "
        "Kullanım Şartları, Gizlilik Politikası veya e-posta linklerini dahil etmeyin. "
        "Yanıtınızı **sadece geçerli JSON** formatında ve tam https URL’leri ile verin.\n\n"
        "Linkler:\n"
    )
    user_prompt += "\n".join(website.links)
    return user_prompt


In [8]:
print(get_links_user_prompt(deponext))

https://www.deponext.com.tr sitesindeki linklerin listesi aşağıdadır. Bu linklerden hangilerinin şirket broşürü için uygun olduğunu seçin, örneğin Hakkımızda, Şirket veya Kariyer sayfaları. Kullanım Şartları, Gizlilik Politikası veya e-posta linklerini dahil etmeyin. Yanıtınızı **sadece geçerli JSON** formatında ve tam https URL’leri ile verin.

Linkler:
https://wa.me/+905373234343
mailto:info@deponext.com
tel:+905373234343
https://www.google.com/maps/place/Cam+Pak+otopark+ve+y%C4%B1kama/@41.0655793,29.0040037,21z/data=!4m14!1m7!3m6!1s0x14cab654199f9f09:0xc56988360e82a6aa!2zR2F5cmV0dGVwZSwgVmVmYWJleSBTb2thxJ_EsSBObzoyNSBEOjEsIDM0MzQ5IEJlxZ9pa3RhxZ8vxLBzdGFuYnVs!3b1!8m2!3d41.0656118!4d29.0042239!3m5!1s0x14cab7e0e90d1005:0x1bfb0e4fa8b47755!8m2!3d41.0656188!4d29.0039287!16s%2Fg%2F11y1dv3j2v?entry=ttu&g_ep=EgoyMDI1MDYxNy4wIKXMDSoASAFQAw%3D%3D
mailto:info@deponext.com
tel:+905373234343
https://www.google.com/maps/place/Cam+Pak+otopark+ve+y%C4%B1kama/@41.0655793,29.0040037,21z/data=!4m14!1m7

In [18]:
def get_links(url):
    website = Website(url)
    user_prompt = get_links_user_prompt(website)
    messages = [
        {"role": "system", "content": link_system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    response = ollama.chat(
        model=MODEL,
        messages=messages,
    )
    content = response["message"]["content"]

    try:
        return json.loads(content)
    except json.JSONDecodeError:
        # JSON parse edilemezse boş dict döndür
        return {"links": []}


In [19]:
get_links("https://www.deponext.com.tr")

{'links': []}

In [20]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_content()
    
    links = get_links(url)
    print("Found links:", links)

    for link in links["links"]:
        result += f"\n\n{link['type']}:\n"
        result += Website(link['url']).get_content()
    
    return result


In [12]:
print(get_all_details("https://www.deponext.com.tr"))

Found links: {'links': [{'type': 'hakkimizda page', 'url': 'https://www.deponext.com.tr/'}, {'type': 'firm page', 'url': 'https://www.deponext.com.tr/'}, {'type': 'careers page', 'url': 'https://www.deponext.com.tr/faq'}]}
Landing page:
 Webpage Title:
Anasayfa | Depo Next - Güvenli Depolama Çözümleri
Webpage Contents:
Eşyalarınız İçin Güvenli ve Kolay Depolama Çözümü
Kendinize Özel Depolama Alanınızı Oluşturun
Teklif Al
DepoNext
Kişiye Özel Depo Kiralama
7/24
Aşama 1
Beklentiler dinlenir,
operasyon hakkında bilgilendirilir.
Depolanacak eşya veya ürünlerin detayına göre operasyon hakkında bilgilendirilir. Eşya veya ürünlerin nasıl taşınacağı, depolama alanı gibi detaylar konuşulur
Aşama 2
İhtiyaca uygun,
teklif verilir.
Depolanacak eşyaların veya ürünlerin durumu göz önünde bulundurularak teklif verilir. Teklifte hem depo ücreti hem de taşıma ücreti belirtilir
Aşama 3
Eşyalar taşınmaya ve,
depolanmaya hazır.
Eşyalar belirtildiği gibi paketlenir ve depoya taşınır. Eşyaların depolanma sü

In [21]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [22]:
def get_brochure_user_promt(company_name, url):
    user_prompt = f"Şirketin adı: {company_name}\n"
    user_prompt += (
        f"Bu şirketin ana sayfası ve diğer ilgili sayfalarının içerikleri aşağıdadır. "
        "Bu bilgileri kullanarak şirket için kısa bir broşür oluştur. "
        "Broşürü Markdown formatında yaz ve yanıtını **Türkçe** ver.\n"
    )
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000]  # 5,000 karakterden uzun ise kısalt
    return user_prompt


In [25]:
get_brochure_user_promt("Deponext","https://www.deponext.com.tr")

Found links: {'links': []}


"Şirketin adı: Deponext\nBu şirketin ana sayfası ve diğer ilgili sayfalarının içerikleri aşağıdadır. Bu bilgileri kullanarak şirket için kısa bir broşür oluştur. Broşürü Markdown formatında yaz ve yanıtını **Türkçe** ver.\nLanding page:\nSayfa Başlığı:\nAnasayfa | Depo Next - Güvenli Depolama Çözümleri\nSayfa İçeriği:\nEşyalarınız İçin Güvenli ve Kolay Depolama Çözümü\nKendinize Özel Depolama Alanınızı Oluşturun\nTeklif Al\nDepoNext\nKişiye Özel Depo Kiralama\n7/24\nAşama 1\nBeklentiler dinlenir,\noperasyon hakkında bilgilendirilir.\nDepolanacak eşya veya ürünlerin detayına göre operasyon hakkında bilgilendirilir. Eşya veya ürünlerin nasıl taşınacağı, depolama alanı gibi detaylar konuşulur\nAşama 2\nİhtiyaca uygun,\nteklif verilir.\nDepolanacak eşyaların veya ürünlerin durumu göz önünde bulundurularak teklif verilir. Teklifte hem depo ücreti hem de taşıma ücreti belirtilir\nAşama 3\nEşyalar taşınmaya ve,\ndepolanmaya hazır.\nEşyalar belirtildiği gibi paketlenir ve depoya taşınır. Eşyal

In [26]:
def create_brochure(company_name,url):
    user_prompt = get_brochure_user_promt(company_name,url)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    response = ollama.chat(
        model=MODEL,
        messages=messages,)
    content=response["message"]["content"]
    display(Markdown(content))

In [27]:
create_brochure("Deponext","https://www.deponext.com.tr")

Found links: {'links': []}


**Deponext - Güvenli Depolama Çözümleri**

**Anasayfa**
DepoNext, Güvenli Depolama Çözümleri sunan bir company olarak, Kundenize personalize depolama çözümü sunuyoruz. İstanbul'un kalbinde bulunan merkezi konumda, kolay erişim imaginable olan our location, eşyelerinize her zaman hızlı ve zahmetsiz bir şekilde erişebilmenizi sağlıyor.

**Müşteri Üzerine**
Binlerce kişiden ve işletmeden Güvenli Depolama ve Taşıma Hizmetleri sunuyoruz. Müşterilerimize %98 oranında müşteri memnuniyeti sağlama missionumuzu ile successful bir müşterilik Politikası kurduk.
-   5.000+ successful transportation
-   12.000+ active warehouse area (m2)
-   25+ city-wide services
-   %98 Customer satisfaction

**Hizmetlerimiz**
-   Kişiye Özel Depolama Kiralama
-   Eşyalarınız.timeless olarak saklanmalarına dedicated storage solutions sunuyoruz.
-   Taşınma Stresi complete olarak ortadan kaldıracaktır. Ekibimiz, ilgisi ve Profesyonellik Harika bir hizmet suniyor.
-   Eşyalarınızı sorunsuz şekilde saklič our experienced team.
-   Güvenli ve Modern Depolama Alanları
-   İstanbul’un dört bir yanındaki ilçelerde güvenli ve modern depolama Hizmeti sunuyoruz.

**Kontak Bilgilerim**
Gayrettepe, Vefabey Sokağı 25/1, 34349 Beşiktaş/İstanbul
info@deponext.com
+90 537 323 43 43

**Depolama Süreci**
-   Beklentilerin dinlenir, operasyon hakkında bilgilendirilir.
-   İhtiyaca uygun, teklif verilir.
-   Eşyalar taşınmaya ve depolanmaya hazır.
-   Eşyalarınızı size özel, oda deponuzda güvende.

**Depolama Alanlarımız**
İstanbul’un dört bir yanındaki ilçelerde Güvenli ve Modern Depolama Hizmeti sunuyoruz. Beşiktaş, İstanbul

**Sıkça Sorulan Sorular**
-  Depolama süreciyle ilgili en çok merak edilen soruların yanıtlarını burada bulabilirsiniz.
-  Eşyalarım nasıl taşınıyor?
-  Depolama süresi ne kadar olabilir?
-  Depo alanları güvenli mi?
-  Depolama ücretleri nasıl belirleniyor?
-  Depolananan eşyaları tekrar nasıl alabilirim?